#  Enhancing accuracy in LLM multi-modal vision use cases on Amazon Bedrock

As Large Language Models (LLMs) evolve to process multiple modalities, we face challenges reminiscent of early text-based LLMs: basic inconsistency, inaccuracy, and hallucinations. 
This notebook focus on techniques to increase the accuracy of vision use cases using various techniques. Inlucded is a quick introduction to AWS Gen AI services landscape and how to get started with using Anthropic Claude and other LLMs on Amazon Bedrock.

## LLM Vision use cases
- Classification (+ few shots)
- Object Detection
- Scene insights extraction (emotions)
- (Structured) OCR
- Content moderation
- Custome tasks - you name it

## Let's deep dive into a specific vision task of Structured OCR

The goal is to A. idetify and read the (Hebrew) text on the page. B. Figure out the structure of the page. C. populate the structure with the text and output it (as Json).
Note that this example includes Hebrew text, some block lettes, some handwritten.

<img title="Sturctured OCR Task" src="./images/structured ocr.png" />

At first I thought about using a specialist AI service like Amazon Textract, but it doesn't support Hebrew (block and hardwritten). Here a multi modal LLM like Anthropic Claude works out of the box.
Let's take a look:


In [5]:
%load_ext autoreload
%autoreload 2
%pip install --upgrade pip
%pip install boto3 --quiet
%pip install botocore --quiet
%pip install json --quiet
import base64
import boto3
import json
from IPython.display import Image
from json_eval import compare_json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [3]:
from llm_util import generate_conversation, get_ocr

base_test = {
                'model_id' : "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
                'input_image' : "./images/pizza-1b.png",
                'instructions_filename' : 'pizza_system_prompt1.txt',
                'temperature' : 0
                }

output = get_ocr(base_test)


Generating message with model us.anthropic.claude-3-5-sonnet-20240620-v1:0
Role: assistant
Text: I'll extract the menu information from the image and structure it according to the provided JSON schema. Here's the structured menu data:

{
  "categories": [
    {
      "name": "תפריט",
      "dishes": [
        {
          "name": "פיצה אישית (S)",
          "price": 20
        },
        {
          "name": "פיצה משפחתית (L)",
          "price": 29
        },
        {
          "name": "פיצה ענקית (XL)",
          "price": 49
        },
        {
          "name": "משפחתית טבעונית",
          "price": 40
        },
        {
          "name": "משפחתית ללא גלוטן",
          "price": 45
        },
        {
          "name": "המלצת השף פיצה יוונית",
          "price": 49,
          "description": "עגבניה,בצל,זיתים שחורים,בולגרית"
        }
      ],
      "extraInfo": [
        "100% צהובה עמק",
        "100% מוצרלה",
        "תוספת שומשום בקצוות הפיצה 2 ₪"
      ]
    },
    {
      "nam

It looks promising, however a manual diff uncovers some mistakes. For example instead of רטבים (sauces) it hallucinated לחמניות (buns).
<img title="Diff view" src="./images/pizza diff.png" />

In [11]:
# It's look pretty good, but let's take evaluate more formally using a similarity score.
# Similarity score is custome function that compares both the json structure and text and numbers values.

import json_eval
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json_eval.extract_json_from_string(output)

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 84.15%

Detailed Difference Report:
Differences found:
  Path: categories[0].dishes[3].description
  Type: Missing Key
  Expected Value 1: V
  Actual Value 2: Not present

  Path: categories[1].dishes[4].description
  Type: String Difference
  Expected Value 1: בתוספת צהובה, מוצרלה ובולגרית
  Actual Value 2: (בתוספת עגבניה, מוצרלה ובולגרית)

  Path: categories[2].extraInfo[0]
  Type: String Difference
  Expected Value 1: כל הסלטים מוגשים עם שתי רטבים לבחירה
  Actual Value 2: כל הסלטים מוגשים עם שמן זית ולימון טבעי

  Path: categories[3].dishes[0].name
  Type: String Difference
  Expected Value 1: תוספת ביצה קשה לסלט
  Actual Value 2: תוספת רוטב פיצה

  Path: categories[4].name
  Type: String Difference
  Expected Value 1: רטבים
  Actual Value 2: תוספות

  Path: categories[4].dishes[1].name
  Type: String Difference
  Expected Value 1: רוטב פיצה, מיני טבסקו
  Actual Value 2: רוטב פיצה, מיני טוסטר



# So we have hallucinations 😱 - now what?
1. What we have out of the box is already very useful, as a human review and fix is faster than typing everything from scratch.
2. If we could detect hallucinations with high probability, this could help us with focusing the human reviewer, and resolving halucinations.
3. Consider alternatives to resolve hallucinations

# How can we detect the likelihood of halucinations in production?
1. Compare outputs of multiple invocations to find variations in json paths values (use high temperature and small prompt changes)
2. Ask the LLM for a confidence score for each value (wishful thinking?)

### Let's try multi-invocations

In [12]:
#model_id = "anthropic.claude-3-sonnet-20240229-v1:0".  # Sonnet 3.5 is far better than 3.0 in this task
base_test = {
                'model_id' : "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
                'input_image' : "./images/pizza-1b.png",
                'instructions_filename' : 'pizza_system_prompt1.txt',
                'temperature' : 0
        }

# variations
from copy import deepcopy
tests = [deepcopy(base_test) for i in range(5)]
tests[0]['temperature'] = 0
tests[1]['temperature'] = 1
tests[2]['temperature'] = 1
tests[3]['temperature'] = 1
tests[4]['temperature'] = 1
#print(tests[0])

In [13]:
def get_outfile_path(idx, t):
        return f'./outputs/pizza_actual_' + t['model_id'] + f"_tempr({t['temperature']})_" + t['instructions_filename'].replace('.txt', '') + f'_{idx}' + '.json'

for idx,test in enumerate(tests): # IMPROVE: can run in parallel
        outfile_path = get_outfile_path(idx, test)
        output = get_ocr(test)
        print(f'Writing output to {outfile_path}')
        open(outfile_path, 'w').write(output)

#actual_json = json_eval.extract_json_from_string(output)

Generating message with model us.anthropic.claude-3-5-sonnet-20240620-v1:0
Role: assistant
Text: I'll extract the menu information from the image and structure it according to the provided JSON schema. Here's the structured menu data:

{
  "categories": [
    {
      "name": "תפריט",
      "dishes": [
        {
          "name": "פיצה אישית (S)",
          "price": 20
        },
        {
          "name": "פיצה משפחתית (L)",
          "price": 29
        },
        {
          "name": "פיצה ענקית (XL)",
          "price": 49
        },
        {
          "name": "משפחתית טבעונית",
          "price": 40
        },
        {
          "name": "משפחתית ללא גלוטן",
          "price": 45
        },
        {
          "name": "המלצת השף פיצה יוונית",
          "price": 49,
          "description": "עגבניה,בצל,זיתים שחורים,בולגרית"
        }
      ],
      "extraInfo": [
        "100% מוצרלה",
        "100% צהובה",
        "תוספת שומשום בקצוות הפיצה 2 ₪"
      ]
    },
    {
      "name": 

In [27]:
print(f'Comparing 5 invocations to ground truth')
for idx,t in enumerate(tests):
    expected = json.load(open('./outputs/pizza_expected.json'))
    actual_output = open(get_outfile_path(idx, t), "r").read()
    actual = json_eval.extract_json_from_string(actual_output)
    
    similarity_score, difference_report = compare_json(expected, actual)
    print(f"Test({idx})<>GT - Similarity: {similarity_score:.2f}%")
    # if i>0:
    #     print(f"Test({idx}) - Detailed Difference Report:")
    #     difference_report.print_report()

print(f'\nComparing 5 invocations to invocation zero as ground')
expected_output = open(get_outfile_path(0, tests[0]), "r").read()
expected = json_eval.extract_json_from_string(expected_output)
for idx,t in enumerate(tests):
    actual_output = open(get_outfile_path(idx, t), "r").read()
    actual = json_eval.extract_json_from_string(actual_output)
    
    similarity_score, difference_report = compare_json(expected, actual)
    print(f"Test({idx})<>Test(0) - Similarity: {similarity_score:.2f}%")
    print(f"Test({idx}) - Detailed Difference Report:")
    difference_report.print_report()

Comparing 5 invocations to ground truth
Test(0)<>GT - Similarity: 73.05%
Test(1)<>GT - Similarity: 81.35%
Test(2)<>GT - Similarity: 83.57%
Test(3)<>GT - Similarity: 87.24%
Test(4)<>GT - Similarity: 77.42%

Comparing 5 invocations to invocation zero as ground
Test(0)<>Test(0) - Similarity: 100.00%
Test(0) - Detailed Difference Report:
No differences found.
Test(1)<>Test(0) - Similarity: 77.45%
Test(1) - Detailed Difference Report:
Differences found:
  Path: categories[0].extraInfo[2]
  Type: String Difference
  Expected Value 1: תוספת שומשום בקצוות הפיצה 2 ₪
  Actual Value 2: תוספת שומשום בקצוות הפיצה: 2 ₪

  Path: categories[0].dishes[5].name
  Type: String Difference
  Expected Value 1: המלצת השף פיצה יוונית
  Actual Value 2: המלצת השף פיצה יווניתת

  Path: categories[1].dishes[0].name
  Type: String Difference
  Expected Value 1: לחם שום אישי
  Actual Value 2: תוספת בצק קשה לסלט

  Path: categories[1].dishes[0].price
  Type: Value Difference
  Expected Value 1: 15
  Actual Value 2: 3

In [14]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))
actual = actual_json

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 81.66%

Detailed Difference Report:
Differences found:
  Path: categories[0].dishes[3].description
  Type: Missing Key
  Value 1: V
  Value 2: Not present

  Path: categories[0].dishes[5].name
  Type: String Difference
  Value 1: המלצת השף פיצה יוונית
  Value 2: המלצת השף פיצה יווניתת

  Path: categories[0].extraInfo[0]
  Type: String Difference
  Value 1: תוספת שימושים בקצוות הפיצה 2 ₪
  Value 2: 100% גבינה עמק

  Path: categories[1].extraInfo[0]
  Type: String Difference
  Value 1: כל הסלטים מגיעים עם שתי לחמניות לבחירה
  Value 2: כל הסלטים מוגשים עם פיתה לבנה

  Path: categories[3].name
  Type: String Difference
  Value 1: רטבים
  Value 2: תוספות

  Path: categories[3].dishes[1].name
  Type: String Difference
  Value 1: רוטב פיצה, מיני סטייק
  Value 2: רוטב פיצה, מיני טוסטר

  Path: categories[4].dishes[4].name
  Type: String Difference
  Value 1: לחם שום ענק (L)
  Value 2: לחם שום הבית (L)

  Path: categories[4].dishes[4].description
  Type: String Difference
  Va

In [ ]:
# Example usage
import json
json1 = json.loads('{"name": "John", "age": 30, "city": "New York", "hobbies": ["reading", "swimming"]}')
json2 = json.loads('{"name": "Jon", "age": 31, "city": "New York", "hobbies": ["reading", "running"]}')

from json_eval import compare_json

similarity_score, difference_report = compare_json(json1, json2)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 77.76%

Detailed Difference Report:
Differences found:
  Path: age
  Type: Value Difference
  Value 1: 30
  Value 2: 31

  Path: name
  Type: String Difference
  Value 1: John
  Value 2: Jon



In [ ]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))
actual = json.load(open(output_filename_fullpath))

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 35.73%

Detailed Difference Report:
Differences found:
  Path: categories[0].extraInfo[0]
  Type: String Difference
  Value 1: תוספת שימושים בקצוות הפיצה 2 ₪
  Value 2: תוספת ביצה קשה לסלט 3 ש"ח

  Path: categories[0].dishes[0].price
  Type: Value Difference
  Value 1: 20
  Value 2: 34

  Path: categories[0].dishes[0].name
  Type: String Difference
  Value 1: פיצה אישית (S)
  Value 2: סלט טורקי

  Path: categories[0].dishes[0].description
  Type: Missing Key
  Value 1: Not present
  Value 2: חסה, מלפפון, עגבניה שרי, חצילים וזיתים שרויים

  Path: categories[0].dishes[1].price
  Type: Value Difference
  Value 1: 29
  Value 2: 34

  Path: categories[0].dishes[1].name
  Type: String Difference
  Value 1: פיצה משפחתית (L)
  Value 2: סלט בגרוזית

  Path: categories[0].dishes[1].description
  Type: Missing Key
  Value 1: Not present
  Value 2: חסה, מלפפון, עגבניה שרי, חצילים וביצים שרויים

  Path: categories[0].dishes[2].price
  Type: Value Difference
  Value 1: 49
  Value 2